## Vault Installation to Minikube via Helm with Integrated Storage

### Start Minikube
- we use minikube to create k8s cluster on your local machine
- start k8s with minikube with command: minikube start
- check cluster status with command: minikube status

### Install the Vault Helm chart
- we use helm chart to install vault
- with helm we have to understand the concept of chart and repo
    - chart: is a collection of k8s manifest which we want to deploy together
    - repo: is a collection of chart
- add repo to helm: 
    - helm repo add $repo_name $repo_address
- update all repo in local to synx with remote repo: 
    - helm repo update
- check chart have been add with command: 
    - helm search repo $chart_name
- write file content to a file using cat command:
    - cat > $file_name << EOF file content ... EOF
- install a chart with helm install command:
    - helm install $release_name $chart_name --values $path_to_config_file.yaml
- list pods in kubectl get command
    - kubectl get pods
- generate root key by using --vault command inside a container
    - run command inside container kubectl exec
        - kubectl exec $pod_name -- $command
    - example:
        - kubectl exec vault-0 -- vault operator init \
            -key-shares=1 \
            -key-threshold=1 \
            -format=json > cluster-keys.json
        - command meaning: run vault operator init command to create root key inside the pod
- read json file with jq cli:
    - jq cli is command line tools which deal with json file
    - jq -r ".unseal_keys_b64[]" cluster-keys.json
- create environment variable to store key
    - VAULT_UNSEAL_KEY=$(jq -r ".unseal_keys_b64[]" cluster-keys.json)
    - to capture return value and then assign to variable
        - variable_name=$(command)
- unseal vault
    - kubectl exec vault-0 -- vault operator unseal $VAULT_UNSEAL_KEY
- add vault-1, vault-2 to raft cluster
    - kubectl exec -ti vault-1 -- vault operator raft join http://vault-0.vault-internal:8200
    - kubectl exec -ti vault-2 -- vault operator raft join http://vault-0.vault-internal:8200
- unseal vault-1, vault-2
    - kubectl exec -ti vault-1 -- vault operator unseal $VAULT_UNSEAL_KEY
    - kubectl exec -ti vault-2 -- vault operator unseal $VAULT_UNSEAL_KEY